# Projet : Passive-Reflector

## Edouard Yvinec et Yoann Coudert--Osmont

Nous rappelons l'objectif du projet : on veut montrer numériquement que la cross-corrélation de signaux émis par des sources bruitées et enregistrées par des récepteurs peuvent servir à localiser un réflecteur.<br> Nous commençons par inclure les bibliothèques dont on aura besoin. Parmi celles-ci se trouve utils qui correspond au .py joint à ce notebook qui contient les fonctions implémentées pour résoudre numériquement le problème.

In [ ]:
%matplotlib inline

import utils as utils
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import warnings
warnings.filterwarnings("ignore")

Dans le projet, on part du même modèle que celui présenté au début du cours avec une solution de l'équation d'onde
$$ \frac{1}{c^2(x)}\frac{\partial^2 u}{\partial t^2} - \Delta_x u = n(t,x) $$
$n$ modélise le champs des bruits de source. Son auto-corrélation vaut
$$ \langle n(t_1,y_1); n(t_2,y_2)\rangle = F(t_2-t_1)\delta(y_1 - y_2)K(y_1) $$
Comme dans le cours, on retrouve la fonction temps-harmonique de Green $G_0$, solution fondamentale du problème de Helmholtz :
$$ \frac{\omega^2}{c^2(x)} \hat G_0 (\omega,x,y) + \Delta_x \hat G_0(\omega,x,y) = - \delta(x - y) $$
Il en suit, 
$$ \hat G(\omega,x,y) = \hat G_0(\omega,x,y) + \sigma_r \omega^2 \hat G_0(\omega,x,z_r) \hat G_0(\omega,z_r,y)$$
Que l'on peut interpréter comme : $\hat G_0(\omega,x,y)$ onde émise de $y$ et reçu en $x$ et $\hat G_0(\omega,x,z) \hat G_0(\omega,z,y)$ est l'onde émise en $y$ réfléchie en $z$ et reçu en $x$. Cette approximation est appelée approximation de Born.

Nous initialisons le problèmes avec les grandeurs que nous utiliserons tout au long de la résolution du problème. Ici $N$ désigne le nombre de sources de positions $y$, étant amenées à émettre, $x$ désigne les positions des récepteurs qui enregistrent les signaux, $c_0$ la vitesse de propagation dans le milieu et $z_r$ la position du réflecteur. La dernière constante $\sigma_r$ sert à résoudre une approximation d'ordre 2 de $\hat G$ détaillée précédemment.

In [ ]:
N = 200
y = utils.sample_ys(N)
x = utils.recevers_xs()
c_0 = 1
z_r = np.array([-5,0,65])
sigma_r = 10**(-3)

In [ ]:
plt.scatter(y[:,2], y[:,0], marker='o', color=(1, 1, 1, 0), edgecolors='r', label='y')
plt.scatter(x[:,2], x[:,0], marker='^', color='g', label='x')
plt.scatter(z_r[2], z_r[0], marker='d', color='b', label=r'$z_r$')
plt.legend()
plt.show()

Chaque source $y_s$ émet un signal $n_s$ i.i.d. gaussien stationnaire de moyenne nulle et de covariance
$$ \langle n_s(t_1); n_s(t_2)\rangle = F(t_1-t_2) $$
Dans notre problème, $\hat F(\omega) = e^{-\omega^2}$. 

----

Dans la première partie, on étudie l'espérance de la cross-correlation $C(\tau,x_1,x_2)$ (aussi obtenu en prenant la cross-correlation sur une durée tendant vers l'infini $T \rightarrow +\infty$) définit par
$$ C(\tau,x_1,x_2) = \frac{1}{2\pi}\int \int K(y)\hat F(\omega) \overline{\hat G(\omega,x_1,y)} \hat G(\omega,x_2,y) e^{-i\omega \tau} dy d\omega $$
En prenant un nombre fini $N$ de sources on obtient l'approximation que l'on note $C_N$, définit par 
$$ C_N(\tau,x_1,x_2) = \frac{1}{2\pi N}\sum_{s =1}^N \int \hat F(\omega) \overline{\hat G(\omega,x_1,y)} \hat G(\omega,x_2,y) e^{-i\omega \tau} d\omega $$

## 1.a) 
On commence par calculer et afficher $$ \tau \rightarrow C_N(\tau, x_5,x_1) $$
pour $\tau \in [-200;200]$.

In [ ]:
tau = np.linspace(-200,200,500)
f = utils.C_N(tau, x[4], x[0], y, c_0, z_r, sigma_r)
plt.plot(tau, f, 'r-')
plt.xlabel(r'$\tau$')
plt.ylabel(r'$C_N$')
plt.title(r'$\tau \rightarrow C_N(\tau,x_5,x_1)$')
plt.show()

Le pic central correspond à la corrélation des ondes arrivant directement aux récepteurs. Mais ce pic masque d'autres pics beaucoup plus faibles correspondant à corrélation d'une onde arrivant à un des récepteurs après la réflection sur $z_r$, avec une onde arrivant directement au second recepteur. Pour les voir on affiche : 
$$ \tau \rightarrow C_N(\tau, x_5,x_1)1_{\mathbb{R}\backslash [-50;50]} $$
sur le meme intervalle.

In [ ]:
f = np.concatenate([f[:200], np.zeros(100), f[-200:]])
plt.plot(tau, f, 'r-')
plt.xlabel(r'$\tau$')
plt.ylabel(r'$C_N$')
plt.title(r'$\tau \rightarrow C_N(\tau,x_5,x_1)1_{\mathbb{R}\backslash[-50;50]}$')
plt.show()

## 1.b)
On calcule ensuite l'image KM $\mathcal{I}_N (y^S)$ avec $y^S$ un candidat à la position du réflecteur. Cette image est alors définit par
$$ \mathcal{I}_N (y^S) = \sum_{k,l =1}^5 C_N \left( \frac{1}{c_0} \left( |x_k - y^S| + |x_l-y^S| \right), x_k, x_l \right) $$

In [ ]:
w_size = 10
dx = np.arange(2*w_size+1) - w_size
y_S = z_r + [0, 0, 1] * dx[:,None] + [1, 0, 0] * dx[:,None,None]
Im = utils.KM(y_S, x, y, c_0, z_r, sigma_r)
plt.imshow(Im)
plt.show()

## 1.c) 
Pour l'analyse de la résolution de l'image nous avons utilisé un résultat du polycopié. En effet, section 2.3.9 : "The resolution analysis of the Reverse-Time imaging function and the Kirch- hoff Migration imaging function goes along the same way as for passive source imaging". Ainsi
$$ R = \frac{N\max I^2}{\|I\|^2} $$

In [ ]:
R = utils.etude_resolution(Im)
print("Resolution de l'image KM de I_N : " + str(R))

On observe tout d'abord que le pic de l'image est plutôt bien centré. Ce pic doit nous donner la position du réflecteur qui est effectivement au centre de notre zone de recherche. La position obtenue indique alors un bon fonctionnement de la méthode. Toutefois cette position varie d'une exécution à l'autre puisque la position des sources varie. A noter que la réparition des sources doit être symétrique par rapport aux droites $(z_r, x_1)$ et $(z_r, x_5)$ pour que la localisation soit la plus précise possible.  

Enfin, la résolution est relativement élevé ce qui indique une probabilité élévé de présence d'un réflecteur et donne la possibilité d'une localisation précise.

-----

Dans ce qui suit nous allons nous intéresser à une seconde approximation de $C$ qui est une approximation de $C_N$, reposant sur le calcul de la cross-correlation empirique (sur une durée finie) $C_{T,N}$ définit par
$$ C_{T,N}(\tau, x_1, x_2) = \frac{1}{T - |\tau|} \int_0^{T-|\tau|} u(t,x_1)u(t+\tau,x_2)dt $$
avec
$$ u(t,x) = \frac{1}{2\pi \sqrt{N}}\sum_{s=1}^N \int \hat G(\omega, x, y_s) \hat n_s(\omega) e^{-i\omega t} d\omega $$

## 2.a
Comme précédemment, on commence par afficher
$$ \tau \rightarrow C_{T,N}(\tau, x_5,x_1) $$
pour $\tau \in [-150;150]$. Mais cette fois-ci pour plusieurs valeurs de $T$.

In [ ]:
tau = np.linspace(-150,150,300)
T_values = [500, 2000, 8000, 32000]
fig, axs = plt.subplots(2, 2, figsize=(15, 12))
for i,T in enumerate(T_values):
    plt.figure(i)
    f = utils.C_TNm(tau, x[4], x[0], T, y, c_0, z_r, sigma_r)
    axs[i//2, i%2].plot(tau, f, 'r-')
    axs[i//2, i%2].set_xlabel(r'$\tau$')
    axs[i//2, i%2].set_title(fr'$\tau \rightarrow C_{{{T},N}}(\tau,x_5,x_1)$')
plt.tight_layout()

Afin d'augmenter la stabilité et puisque le résultat obtenu dépend du tirage de $n_s$, on peut considérer $M$ tirages de $C_{T,N}$ i.i.d. et ainsi étudier
$$ C_{T,N,M} = \frac{1}{M}\sum_{m=1}^M C_{T,N}^{(m)} $$

In [ ]:
T, M = 10000, 18
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
f = utils.C_TNM(M, tau, x[4], x[0], T, y, c_0, z_r, sigma_r)
axs[0].plot(tau, f, 'r-')
axs[0].set_xlabel(r'$\tau$')
axs[0].set_title(fr'$\tau \rightarrow C_{{{T},N,{M}}}(\tau,x_5,x_1)$')
f = utils.C_N(tau, x[4], x[0], y, c_0, z_r, sigma_r)
axs[1].plot(tau, f, 'r-')
axs[1].set_xlabel(r'$\tau$')
axs[1].set_title(r'$\tau \rightarrow C_N(\tau,x_5,x_1)$')
plt.show()

On retrouve bien une valeur approchée de l'espérance de la cross-correlation (à droite). Cependant, sur l'ensemble $\mathbb{R}\backslash[-50;50]$ la nouvelle approximation est très bruitée. Les pics dus à la corrélation d'une onde direct avec une onde réfléchie sont alors indiscernables et il n'y a alors plus vraiment espoir de trouver le réflecteur. On peut aussi observé un facteur multiplicatif d'environ 1.5 entre les deux signaux. Une constante a probablement été oubliée quelque part dans le code.

On peut aussi constater que les pics dus à la corrélation entre une onde directe et une onde réfléchie sont bien calculés mais cachés par le bruit de la corrélation entre deux ondes directs. Pour cela pour le calcul de l'onde reçu en $x_1$, il suffit de prendre la fonction de Green suivante :
$$ \hat G(\omega,x,y) = \sigma_r \omega^2 \hat G_0(\omega,x,z_r) \hat G_0(\omega,z_r,y) $$
qui ne prend en compte que la réflexion, et pour $x_5$ on prend la fonction de Green suivante :
$$ \hat G(\omega,x,y) = \hat G_0(\omega,x,y) $$
qui ne prend en compte que les ondes directes. On calcule alors la corrélation en prenant ces fonctions de Green ci, puis on somme avec la corrélation obtenu en inversant les fonctions de Green entre $x_1$ et $x_5$. On obtient alors le graphique suivant :

In [ ]:
T, M = 10000, 18
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
f = utils.C_TNM(M, tau, x[4], x[0], T, y, c_0, z_r, sigma_r, non_direct=True)
axs[0].plot(tau, f, 'r-')
axs[0].set_xlabel(r'$\tau$')
axs[0].set_title(fr'$\tau \rightarrow C_{{{T},N,{M}}}(\tau,x_5,x_1)$ (ondes direct + réfléchie)')
f = utils.C_N(tau, x[4], x[0], y, c_0, z_r, sigma_r)
f[abs(tau) < 50] = 0
axs[1].plot(tau, f, 'r-')
axs[1].set_xlabel(r'$\tau$')
axs[1].set_title(r'$\tau \rightarrow C_N(\tau,x_5,x_1)1_{\mathbb{R}\backslash[-50;50]}$')
plt.show()

On observe alors à nouveau exactement les mêmes pics mais avec un peu de bruit autour. Hélas ces pics sont 5 ordres de grandeur en dessous du bruit de la corrélation entre les ondes directs et sont donc impossible à observer dans le signal total.

## 2.b
Afin d'étudier la qualité des résultats obtenus par cette seconde approche, on calcule ensuite l'image KM $\mathcal{I}_{T,N,M} (y^S)$ avec $y^S$ un candidat à la position du réflecteur. Cette image est alors définit par
$$ \mathcal{I}_{T,N,M} (y^S) = \sum_{k,l =1}^5 C_{T,N,M}(|x_k - y^S| + |x_l-y^S|, x_k, x_l) $$

In [ ]:
w_size = 10
dx = np.arange(2*w_size+1) - w_size
y_S = z_r + [0, 0, 1] * dx[:,None] + [1, 0, 0] * dx[:,None,None]
Im = utils.KMT(y_S, x, y, 10000, 20, c_0, z_r, sigma_r)
plt.imshow(Im)
plt.show()

On obtient une image qui varie beaucoup entre plusieurs exécutions. En effet les valeurs de $\tau$ utilisées pour calculer cette image se trouvent dans l'intervalle $[50, 115]$, où l'on observe que du bruit dépendant de l'exécution. Il faudrait donc augmenter $T$ et $M$ pour réduire ce bruit et espérer observer une image correcte mais on serait trop vite bloqué par un temps de calcul qui deviendrait beaucoup trop long. Où alors il faudrait revoir la façon de calculer l'image, la compléxité étant en :
$$  \mathcal{O}(T \cdot M \cdot (N + \log T + n_{pixels})) $$
où $n_{pixels}$ est le nombre de pixels dans l'image.

## 2.c
Malgré le fait que l'image précédente ne permette pas de localiser le réflecteur, on calcule tout de même la résolution de l'image.

In [ ]:
R = utils.etude_resolution(Im)
print("Resolution de l'image KM de I_NT: " + str(R))

Variant énormément d'une exécution à l'autre il est difficile d'étudier la stabilité de cette image selon $T$ et $M$.